In [ ]:
!pip install -U git+https://github.com/mljar/mljar-supervised.git@master
!pip install -U matplotlib==3.1.3

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from supervised.automl import AutoML # mljar-supervised


In [ ]:
train=pd.read_csv("../input/tabular-playground-series-jan-2021/train.csv")
test=pd.read_csv("../input/tabular-playground-series-jan-2021/test.csv")

In [ ]:

# Load the data
X = train.drop(columns=['target'])
# sc = StandardScaler()
# x=sc.fit_transform(x)
y = train['target'].values
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=123,
)



In [ ]:
# train models with AutoML
automl = AutoML( algorithms=["CatBoost", "Xgboost", "LightGBM","Neural Network"],
    model_time_limit=60*60,
    start_random_models=10,
    hill_climbing_steps=3,
    top_models_to_improve=3,
    golden_features=True,
    features_selection=True,
    stack_models=True,
    train_ensemble=True,
    explain_level=0,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 5,
        "shuffle": False,
        "stratify": True,
    })
automl.fit(X, y)



In [ ]:
# compute the MSE on test data
predictions = automl.predict(X_test)
print("Test MSE:", mean_squared_error(y_test, predictions,squared=False))

In [ ]:
preds = automl.predict(test)
submission = test[['id']]
submission['target'] = preds
submission.to_csv('submission.csv', index = False)